# 데이터 준비

In [1]:
import pandas as pd

def make_two_sents(file_name):
    df = pd.read_csv(file_name, error_bad_lines=False,  sep='\t')
    df = df.dropna() # 널 제거
    return df

train_file_path = '../data/KorSTS/sts-train.tsv'
test_file_path = '../data/KorSTS/sts-test.tsv'

train_df = make_two_sents(file_name= train_file_path)
test_df = make_two_sents(file_name= test_file_path)
train_df.head()

b'Skipping line 2096: expected 7 fields, saw 8\nSkipping line 2511: expected 7 fields, saw 8\nSkipping line 2652: expected 7 fields, saw 8\nSkipping line 2729: expected 7 fields, saw 8\nSkipping line 2818: expected 7 fields, saw 8\nSkipping line 3063: expected 7 fields, saw 8\nSkipping line 3385: expected 7 fields, saw 8\n'


,genre,filename,year,id,score,sentence1,sentence2
0,main-captions,MSRvid,2012test,1,5.00,비행기가 이륙하고 있다.,비행기가 이륙하고 있다.
1,main-captions,MSRvid,2012test,4,3.80,한 남자가 큰 플루트를 연주하고 있다.,남자가 플루트를 연주하고 있다.
2,main-captions,MSRvid,2012test,5,3.80,한 남자가 피자에 치즈를 뿌려놓고 있다.,한 남자가 구운 피자에 치즈 조각을 뿌려놓고 있다.
3,main-captions,MSRvid,2012test,6,2.60,세 남자가 체스를 하고 있다.,두 남자가 체스를 하고 있다.
4,main-captions,MSRvid,2012test,9,4.25,한 남자가 첼로를 연주하고 있다.,자리에 앉은 남자가 첼로를 연주하고 있다.


In [2]:
print(train_df.info())
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5691 entries, 0 to 5695
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   genre      5691 non-null   object 
 1   filename   5691 non-null   object 
 2   year       5691 non-null   object 
 3   id         5691 non-null   int64  
 4   score      5691 non-null   float64
 5   sentence1  5691 non-null   object 
 6   sentence2  5691 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 355.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1376 entries, 0 to 1378
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   genre      1376 non-null   object 
 1   filename   1376 non-null   object 
 2   year       1376 non-null   object 
 3   id         1376 non-null   int64  
 4   score      1376 non-null   float64
 5   sentence1  1376 non-null   object 
 6   sentence2  1376 non-null   object 

In [3]:
train_df[train_df['score'] > 3].head()

,genre,filename,year,id,score,sentence1,sentence2
0,main-captions,MSRvid,2012test,1,5.00,비행기가 이륙하고 있다.,비행기가 이륙하고 있다.
1,main-captions,MSRvid,2012test,4,3.80,한 남자가 큰 플루트를 연주하고 있다.,남자가 플루트를 연주하고 있다.
2,main-captions,MSRvid,2012test,5,3.80,한 남자가 피자에 치즈를 뿌려놓고 있다.,한 남자가 구운 피자에 치즈 조각을 뿌려놓고 있다.
4,main-captions,MSRvid,2012test,9,4.25,한 남자가 첼로를 연주하고 있다.,자리에 앉은 남자가 첼로를 연주하고 있다.
5,main-captions,MSRvid,2012test,11,4.25,몇몇 남자들이 싸우고 있다.,두 남자가 싸우고 있다.


In [4]:
train_df[train_df['score'] < 3].head()

,genre,filename,year,id,score,sentence1,sentence2
3,main-captions,MSRvid,2012test,6,2.600,세 남자가 체스를 하고 있다.,두 남자가 체스를 하고 있다.
6,main-captions,MSRvid,2012test,12,0.500,남자가 담배를 피우고 있다.,남자가 스케이트를 타고 있다.
7,main-captions,MSRvid,2012test,13,1.600,남자가 피아노를 치고 있다.,남자가 기타를 연주하고 있다.
8,main-captions,MSRvid,2012test,14,2.200,한 남자가 기타를 치고 노래를 부르고 있다.,한 여성이 어쿠스틱 기타를 연주하고 노래를 부르고 있다.
14,main-captions,MSRvid,2012test,21,1.667,한 남자가 도로를 달리고 있다.,판다 개가 도로에서 달리고 있다.


## 데이터 전처리

In [5]:
import numpy as np
import os
def preprocess(data, new_file_path, b_train=False ):
    df = data.copy()
    df = df.dropna() # 널 제거    
    df['label'] = np.where(df['score'] >= 3, 1, 0) # score 가 3 이상이면 레이블 1, 아니면 0
    if b_train:
        # print("drop noise row")
        df = df.drop([2902, 3206, 3258, 3379])        # noise 데이터 삭제
    
    df.to_csv(new_file_path, sep='\t', index=None)        
    return df

preproc_train_dir = 'preproc/train'
preproc_test_dir = 'preproc/test'
os.makedirs(preproc_train_dir, exist_ok=True)   
os.makedirs(preproc_test_dir, exist_ok=True)   

train_file_path = os.path.join(preproc_train_dir, 'train.tab')
test_file_path = os.path.join(preproc_test_dir, 'test.tab')

train_p_df = preprocess(train_df, train_file_path, b_train=True)    
test_p_df = preprocess(test_df, test_file_path)    
train_p_df
test_p_df
                 

,genre,filename,year,id,score,sentence1,sentence2,label
0,main-captions,MSRvid,2012test,24,2.5,한 소녀가 머리를 스타일링하고 있다.,한 소녀가 머리를 빗고 있다.,0
1,main-captions,MSRvid,2012test,33,3.6,한 무리의 남자들이 해변에서 축구를 한다.,한 무리의 소년들이 해변에서 축구를 하고 있다.,1
2,main-captions,MSRvid,2012test,45,5.0,한 여성이 다른 여성의 발목을 재고 있다.,한 여자는 다른 여자의 발목을 측정한다.,1
3,main-captions,MSRvid,2012test,63,4.2,한 남자가 오이를 자르고 있다.,한 남자가 오이를 자르고 있다.,1
4,main-captions,MSRvid,2012test,66,1.5,한 남자가 하프를 연주하고 있다.,한 남자가 키보드를 연주하고 있다.,0
...,...,...,...,...,...,...,...,...
1374,main-news,headlines,2016,1354,0.0,필리핀과 캐나다는 더 나아가 관계를 증진시키겠다고 약속한다.,필리핀은 여객선 침몰 후 100명을 구한다.,0
1375,main-news,headlines,2016,1360,1.0,이스라엘은 팔레스타인인의 예루살렘의 구 시가지 출입을 금지한다.,팔레스타인인과 이스라엘 두 국가 해결안은 그림의 떡,0
1376,main-news,headlines,2016,1368,1.0,비밀경호국에 대해 얼마나 알고 계십니까?,양측 의원들은 비밀경호국에 분노를 표출한다,0
1377,main-news,headlines,2016,1420,0.0,이란 회담이 재개되면서 오바마는 사우디의 두려움을 달래기 위해 고군분투하고 있다.,미얀마는 일요일 투표를 위한 유권자 명단을 확정하기 위해 고군분투하고 있다.,0
